In [1]:
!pip install ccxt

DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/dill-0.3.9-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/looseversion-1.3.0-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/lightning_thunder-0.2.2.dev0-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/lightning_utilities-0.14.0-py3.12.egg is deprecated. pip 25.1 w

In [5]:
import ccxt
import pandas as pd
import time

# Инициализация биржи
exchange = ccxt.binance()
symbol = 'BTC/USDT'
timeframe = '4h'
since = exchange.parse8601('2017-08-17T00:00:00Z')  # Самая ранняя дата для BTC/USDT на Binance

# Список для хранения всех данных
all_ohlcv = []

# Текущая временная метка в миллисекундах
current_time = int(time.time() * 1000)  # Текущая дата в миллисекундах

# Ограничение на количество свечей за запрос (Binance обычно возвращает до 500 или 1000)
limit = 1000

# Загрузка данных с пагинацией
while since < current_time:
    try:
        ohlcv = exchange.fetch_ohlcv(symbol, timeframe, since, limit)
        if not ohlcv:
            break  # Если данных больше нет, выходим
        all_ohlcv.extend(ohlcv)
        since = ohlcv[-1][0] + 1  # Обновляем since до следующей свечи
        print(f"Загружено {len(ohlcv)} свечей до {pd.to_datetime(since, unit='ms')}")
        time.sleep(1)  # Задержка, чтобы не превысить лимит запросов API
    except Exception as e:
        print(f"Ошибка: {e}")
        break

# Преобразование в DataFrame
df = pd.DataFrame(all_ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
df.set_index('timestamp', inplace=True)

# Удаление дубликатов, если есть
df = df[~df.index.duplicated(keep='first')]

# Сохранение в CSV
df.to_csv('btc_usdt_4h_full.csv')
print(f"Данные сохранены в 'btc_usdt_4h_full.csv'. Всего строк: {len(df)}")

Загружено 1000 свечей до 2018-01-30 16:00:00.001000
Загружено 1000 свечей до 2018-07-18 00:00:00.001000
Загружено 1000 свечей до 2018-12-31 20:00:00.001000
Загружено 1000 свечей до 2019-06-17 00:00:00.001000
Загружено 1000 свечей до 2019-11-30 20:00:00.001000
Загружено 1000 свечей до 2020-05-15 16:00:00.001000
Загружено 1000 свечей до 2020-10-29 08:00:00.001000
Загружено 1000 свечей до 2021-04-14 00:00:00.001000
Загружено 1000 свечей до 2021-09-27 16:00:00.001000
Загружено 1000 свечей до 2022-03-13 08:00:00.001000
Загружено 1000 свечей до 2022-08-27 00:00:00.001000
Загружено 1000 свечей до 2023-02-09 16:00:00.001000
Загружено 1000 свечей до 2023-07-26 08:00:00.001000
Загружено 1000 свечей до 2024-01-09 00:00:00.001000
Загружено 1000 свечей до 2024-06-23 16:00:00.001000
Загружено 1000 свечей до 2024-12-07 08:00:00.001000
Загружено 685 свечей до 2025-03-31 12:00:00.001000
Данные сохранены в 'btc_usdt_4h_full.csv'. Всего строк: 16685


In [5]:
!pip install pandas_ta

DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/dill-0.3.9-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/looseversion-1.3.0-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/lightning_thunder-0.2.2.dev0-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/lightning_utilities-0.14.0-py3.12.egg is deprecated. pip 25.1 w

In [11]:
import pandas as pd
import pandas_ta as ta

# Загрузка данных
df = pd.read_csv('btc_usdt_4h_full.csv', index_col='timestamp', parse_dates=True)

# Differencing
for k in range(1, 6):
    for col in ['open', 'high', 'low', 'close', 'volume']:
        df[f'{col}_diff_{k}'] = df[col] / df[col].shift(k)

# EBSW
ebsw_result = ta.ebsw(df['close'])
df['ebsw'] = ebsw_result

# Chaikin Money Flow
df['cmf'] = ta.cmf(df['high'], df['low'], df['close'], df['volume'], length=20)

# VWAP
df['tp'] = (df['low'] + df['high'] + df['close']) / 3
df['vwap'] = (df['tp'] * df['volume']).cumsum() / df['volume'].cumsum()
df['vwap_open'] = df['vwap'] / df['open']

# Other Ratios
df['high_low'] = df['high'] / df['low']
df['high_open'] = df['high'] / df['open']
df['close_open'] = df['close'] / df['open']
df['low_open'] = df['low'] / df['open']

# Time Information
df['hour'] = df.index.hour
df['day'] = df.index.day
df['month'] = df.index.month

# Удаляем строки с NaN
df.dropna(inplace=True)

df['high_next'] = df['high'].shift(-1)
df['low_next'] = df['low'].shift(-1)
df['u_t1'] = (df['high_next'] - df['close']) / df['close']
df['v_t1'] = (df['low_next'] - df['close']) / df['close']

def classify(u, v):
    if u >= 0.0075:
        return 0  # c0
    elif v <= -0.0075:
        return 1  # c1
    else:
        return 2  # c2

df['target'] = [classify(u, v) for u, v in zip(df['u_t1'], df['v_t1'])]
df.dropna(inplace=True)  # Удаляем последнюю строку

# Сохранение обработанных данных
df.to_csv('btc_usdt_4h_with_target.csv')

/tmp/ipykernel_157/429230575.py:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  ebsw_result = ta.ebsw(df['close'])


In [2]:
!pip install imblearn

DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/dill-0.3.9-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/looseversion-1.3.0-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/lightning_thunder-0.2.2.dev0-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/lightning_utilities-0.14.0-py3.12.egg is deprecated. pip 25.1 w

In [6]:
from pytorch_tabnet.tab_model import TabNetClassifier
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, classification_report
import pandas_ta as ta
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Загрузка и обработка данных
df = pd.read_csv('btc_usdt_4h_with_target.csv', index_col='timestamp', parse_dates=True)
df.fillna(method='ffill', inplace=True)
df.fillna(0, inplace=True)

# Пересчет целевой переменной с новым порогом
df['high_next'] = df['high'].shift(-1)
df['low_next'] = df['low'].shift(-1)
df['u_t1'] = (df['high_next'] - df['close']) / df['close']
df['v_t1'] = (df['low_next'] - df['close']) / df['close']
threshold = 0.005
df['target'] = np.where(df['u_t1'] >= threshold, 0, np.where(df['v_t1'] <= -threshold, 1, 2))

# Добавление ATR
def calculate_atr(df, period=14):
    df['tr'] = np.maximum(df['high'] - df['low'], 
                         np.maximum(abs(df['high'] - df['close'].shift(1)), 
                                    abs(df['low'] - df['close'].shift(1))))
    df['atr'] = df['tr'].rolling(window=period).mean()
    return df

df = calculate_atr(df)
df['cmf'] = ta.cmf(df['high'], df['low'], df['close'], df['volume'], length=20)
df['vwap'] = ta.vwap(df['high'], df['low'], df['close'], df['volume'])
df['vwap_open'] = df['vwap'] / df['open']

# Разделение данных
train_df = df['2017-08-23 16:00:00':'2021-12-31 16:00:00']
val_df = df['2022-01-01 00:00:00':'2022-02-13 20:00:00']
test_df = df['2022-02-14 00:00:00':'2022-04-23 00:00:00']

# Ограничение признаков
# key_features = ['open', 'high', 'low', 'close', 'volume', 'close_open', 'high_low', 
#                 'open_diff_1', 'open_diff_2', 'close_diff_1', 'close_diff_2', 
#                 'volume_diff_1', 'volume_diff_2', 'cmf', 'vwap_open', 'atr']
key_features = ['open', 'high', 'low', 'close', 'volume', 'close_open', 'high_low', 
                'open_diff_1', 'open_diff_2', 'close_diff_1', 'close_diff_2', 
                'volume_diff_1', 'volume_diff_2', 'cmf', 'vwap_open', 'atr']
features = [col for col in df.columns if col in key_features]
X_train = train_df[features].values
y_train = train_df['target'].values
X_val = val_df[features].values
y_val = val_df['target'].values
X_test = test_df[features].values
y_test = test_df['target'].values

# Обработка inf
for i, col in enumerate(features):
    max_val = np.percentile(df[col][~np.isinf(df[col])], 99)
    min_val = np.percentile(df[col][~np.isinf(df[col])], 1)
    X_train[:, i] = np.where(np.isinf(X_train[:, i]), max_val, X_train[:, i])
    X_val[:, i] = np.where(np.isinf(X_val[:, i]), max_val, X_val[:, i])
    X_test[:, i] = np.where(np.isinf(X_test[:, i]), max_val, X_test[:, i])
X_train = np.nan_to_num(X_train, nan=0.0)
X_val = np.nan_to_num(X_val, nan=0.0)
X_test = np.nan_to_num(X_test, nan=0.0)

# Нормализация и шум
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train) + np.random.normal(0, 0.005, X_train.shape)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

# Проверка распределения
print("Распределение y_train:", np.bincount(y_train))
print("Распределение y_val:", np.bincount(y_val))
print("Распределение y_test:", np.bincount(y_test))

# Веса классов
class_weights_dict = {0: 1.0, 1: 1.2, 2: 2.0}
print("Веса классов:", class_weights_dict)

# Инициализация и обучение TabNet
clf = TabNetClassifier(
    n_d=128, n_a=128, n_steps=5, gamma=1.5,
    lambda_sparse=0.005, optimizer_params=dict(lr=1e-3, weight_decay=1e-5),
    mask_type='sparsemax', n_shared=2,
    verbose=1, seed=42
)
clf.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    eval_metric=['accuracy'],
    max_epochs=300,
    patience=100,
    batch_size=512,
    virtual_batch_size=256,
    weights=class_weights_dict,
    drop_last=True
)

# Предсказание и оценка
y_pred = clf.predict(X_test)
print("Распределение предсказанных классов:", np.bincount(y_pred))
print("Распределение истинных классов:", np.bincount(y_test))
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')
print(f"Accuracy: {accuracy:.4f}")
print(f"F1-score: {f1:.4f}")
print("\nClassification Report:\n", classification_report(y_test, y_pred, target_names=['c0', 'c1', 'c2']))

# Важность признаков
feature_importances = pd.Series(clf.feature_importances_, index=features)
print("\nТоп-5 важных признаков:\n", feature_importances.sort_values(ascending=False).head())

/tmp/ipykernel_143/607870151.py:10: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', inplace=True)
/usr/local/lib/python3.12/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


Распределение y_train: [6147 2598  786]
Распределение y_val: [167  88   9]
Распределение y_test: [245 125  39]
Веса классов: {0: 1.0, 1: 1.2, 2: 2.0}
epoch 0  | loss: 1.9861  | val_0_accuracy: 0.58712 |  0:00:01s
epoch 1  | loss: 1.22394 | val_0_accuracy: 0.57955 |  0:00:02s
epoch 2  | loss: 1.05896 | val_0_accuracy: 0.58712 |  0:00:03s
epoch 3  | loss: 1.03916 | val_0_accuracy: 0.49621 |  0:00:04s
epoch 4  | loss: 1.01917 | val_0_accuracy: 0.59091 |  0:00:04s
epoch 5  | loss: 0.99369 | val_0_accuracy: 0.5     |  0:00:05s
epoch 6  | loss: 0.96717 | val_0_accuracy: 0.625   |  0:00:06s
epoch 7  | loss: 0.95142 | val_0_accuracy: 0.61742 |  0:00:07s
epoch 8  | loss: 0.92837 | val_0_accuracy: 0.57955 |  0:00:08s
epoch 9  | loss: 0.93484 | val_0_accuracy: 0.63258 |  0:00:08s
epoch 10 | loss: 0.94141 | val_0_accuracy: 0.64773 |  0:00:09s
epoch 11 | loss: 0.92336 | val_0_accuracy: 0.60606 |  0:00:10s
epoch 12 | loss: 0.92785 | val_0_accuracy: 0.49242 |  0:00:11s
epoch 13 | loss: 0.89574 | val_

/usr/local/lib/python3.12/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Распределение предсказанных классов: [376  32   1]
Распределение истинных классов: [245 125  39]
Accuracy: 0.5795
F1-score: 0.4788

Classification Report:
               precision    recall  f1-score   support

          c0       0.60      0.92      0.73       245
          c1       0.34      0.09      0.14       125
          c2       0.00      0.00      0.00        39

    accuracy                           0.58       409
   macro avg       0.31      0.34      0.29       409
weighted avg       0.47      0.58      0.48       409


Топ-5 важных признаков:
 atr            0.192343
high_low       0.123292
low            0.092999
high           0.088348
open_diff_2    0.071955
dtype: float64


In [17]:
from pytorch_tabnet.tab_model import TabNetClassifier
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, classification_report
from imblearn.over_sampling import SMOTE
import pandas_ta as ta
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Загрузка и обработка данных
df = pd.read_csv('btc_usdt_4h_with_target.csv', index_col='timestamp', parse_dates=True)
df.fillna(method='ffill', inplace=True)
df.fillna(0, inplace=True)

# Пересчет целевой переменной
df['high_next'] = df['high'].shift(-1)
df['low_next'] = df['low'].shift(-1)
df['u_t1'] = (df['high_next'] - df['close']) / df['close']
df['v_t1'] = (df['low_next'] - df['close']) / df['close']
threshold = 0.006
df['target'] = np.where(df['u_t1'] >= threshold, 0, np.where(df['v_t1'] <= -threshold, 1, 2))

# Добавление признаков
def calculate_atr(df, period=14):
    df['tr'] = np.maximum(df['high'] - df['low'], 
                         np.maximum(abs(df['high'] - df['close'].shift(1)), 
                                    abs(df['low'] - df['close'].shift(1))))
    df['atr'] = df['tr'].rolling(window=period).mean()
    return df

df = calculate_atr(df)
df['cmf'] = ta.cmf(df['high'], df['low'], df['close'], df['volume'], length=20)
df['vwap'] = ta.vwap(df['high'], df['low'], df['close'], df['volume'])
df['vwap_open'] = df['vwap'] / df['open']
df['rsi'] = ta.rsi(df['close'], length=14)
bb = ta.bbands(df['close'], length=20)
df['bb_width'] = (bb['BBU_20_2.0'] - bb['BBL_20_2.0']) / df['close']
df['volume_change'] = df['volume'].pct_change()

# Разделение данных
train_df = df['2017-08-23 16:00:00':'2021-12-31 16:00:00']
val_df = df['2022-01-01 00:00:00':'2022-02-13 20:00:00']
test_df = df['2022-02-14 00:00:00':'2022-04-23 00:00:00']

# Ограничение признаков
key_features = ['open', 'high', 'low', 'close', 'volume', 'close_open', 'high_low', 
                'open_diff_1', 'open_diff_2', 'close_diff_1', 'close_diff_2', 
                'volume_diff_1', 'volume_diff_2', 'cmf', 'vwap_open', 'atr', 
                'rsi', 'bb_width', 'volume_change']
features = [col for col in df.columns if col in key_features]
X_train = train_df[features].values
y_train = train_df['target'].values
X_val = val_df[features].values
y_val = val_df['target'].values
X_test = test_df[features].values
y_test = test_df['target'].values

# Обработка inf и NaN
for i, col in enumerate(features):
    max_val = np.percentile(df[col][~np.isinf(df[col])], 99)
    min_val = np.percentile(df[col][~np.isinf(df[col])], 1)
    X_train[:, i] = np.where(np.isinf(X_train[:, i]), max_val, X_train[:, i])
    X_val[:, i] = np.where(np.isinf(X_val[:, i]), max_val, X_val[:, i])
    X_test[:, i] = np.where(np.isinf(X_test[:, i]), max_val, X_test[:, i])
X_train = np.nan_to_num(X_train, nan=0.0)
X_val = np.nan_to_num(X_val, nan=0.0)
X_test = np.nan_to_num(X_test, nan=0.0)

# Нормализация и шум
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train) + np.random.normal(0, 0.005, X_train.shape)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

# Oversampling с SMOTE
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

# Проверка распределения
print("Распределение y_train (после SMOTE):", np.bincount(y_train_res))
print("Распределение y_val:", np.bincount(y_val))
print("Распределение y_test:", np.bincount(y_test))

# Веса классов
class_weights_dict = {0: 1.0, 1: 1.5, 2: 5.0}
print("Веса классов:", class_weights_dict)

# Инициализация и обучение TabNet
clf = TabNetClassifier(
    n_d=128, n_a=128, n_steps=5, gamma=1.5,
    lambda_sparse=0.001, optimizer_params=dict(lr=2e-3, weight_decay=1e-5),
    mask_type='sparsemax', n_shared=2,
    verbose=1, seed=42
)
clf.fit(
    X_train_res, y_train_res,
    eval_set=[(X_val, y_val)],
    eval_metric=['accuracy', 'balanced_accuracy'],
    max_epochs=300,
    patience=50,
    batch_size=512,
    virtual_batch_size=256,
    weights=class_weights_dict,
    drop_last=True
)

# Предсказание и оценка
y_pred = clf.predict(X_test)
print("Распределение предсказанных классов:", np.bincount(y_pred))
print("Распределение истинных классов:", np.bincount(y_test))
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')
print(f"Accuracy: {accuracy:.4f}")
print(f"F1-score: {f1:.4f}")
print("\nClassification Report:\n", classification_report(y_test, y_pred, target_names=['c0', 'c1', 'c2']))

# Важность признаков
feature_importances = pd.Series(clf.feature_importances_, index=features)
print("\nТоп-5 важных признаков:\n", feature_importances.sort_values(ascending=False).head())

/tmp/ipykernel_143/1219255241.py:11: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='ffill', inplace=True)
/usr/local/lib/python3.12/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


Распределение y_train (после SMOTE): [5519 5519 5519]
Распределение y_val: [149  92  23]
Распределение y_test: [219 126  64]
Веса классов: {0: 1.0, 1: 1.5, 2: 5.0}
epoch 0  | loss: 1.34181 | val_0_accuracy: 0.14773 | val_0_balanced_accuracy: 0.37606 |  0:00:01s
epoch 1  | loss: 0.96968 | val_0_accuracy: 0.24621 | val_0_balanced_accuracy: 0.35814 |  0:00:03s
epoch 2  | loss: 0.94348 | val_0_accuracy: 0.28409 | val_0_balanced_accuracy: 0.40662 |  0:00:04s
epoch 3  | loss: 0.87253 | val_0_accuracy: 0.26515 | val_0_balanced_accuracy: 0.33854 |  0:00:05s
epoch 4  | loss: 0.83187 | val_0_accuracy: 0.28409 | val_0_balanced_accuracy: 0.3557  |  0:00:07s
epoch 5  | loss: 0.79475 | val_0_accuracy: 0.31061 | val_0_balanced_accuracy: 0.35612 |  0:00:08s
epoch 6  | loss: 0.77439 | val_0_accuracy: 0.32955 | val_0_balanced_accuracy: 0.43624 |  0:00:10s
epoch 7  | loss: 0.73645 | val_0_accuracy: 0.37121 | val_0_balanced_accuracy: 0.42058 |  0:00:11s
epoch 8  | loss: 0.70939 | val_0_accuracy: 0.34091 |

/usr/local/lib/python3.12/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Распределение предсказанных классов: [ 72 198 139]
Распределение истинных классов: [219 126  64]
Accuracy: 0.3521
F1-score: 0.3391

Classification Report:
               precision    recall  f1-score   support

          c0       0.62      0.21      0.31       219
          c1       0.29      0.46      0.36       126
          c2       0.29      0.64      0.40        64

    accuracy                           0.35       409
   macro avg       0.40      0.44      0.36       409
weighted avg       0.47      0.35      0.34       409


Топ-5 важных признаков:
 high_low    0.185959
atr         0.145906
close       0.128660
volume      0.067386
open        0.061043
dtype: float64


In [18]:
from pytorch_tabnet.tab_model import TabNetClassifier
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, classification_report
from imblearn.over_sampling import SMOTE
import pandas_ta as ta
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Загрузка и обработка данных
df = pd.read_csv('btc_usdt_4h_with_target.csv', index_col='timestamp', parse_dates=True)
df.ffill(inplace=True)
df.fillna(0, inplace=True)

# Пересчет целевой переменной
df['high_next'] = df['high'].shift(-1)
df['low_next'] = df['low'].shift(-1)
df['u_t1'] = (df['high_next'] - df['close']) / df['close']
df['v_t1'] = (df['low_next'] - df['close']) / df['close']
threshold = 0.006
df['target'] = np.where(df['u_t1'] >= threshold, 0, np.where(df['v_t1'] <= -threshold, 1, 2))

# Добавление признаков
def calculate_atr(df, period=14):
    df['tr'] = np.maximum(df['high'] - df['low'], 
                         np.maximum(abs(df['high'] - df['close'].shift(1)), 
                                    abs(df['low'] - df['close'].shift(1))))
    df['atr'] = df['tr'].rolling(window=period).mean()
    return df

df = calculate_atr(df)
df['cmf'] = ta.cmf(df['high'], df['low'], df['close'], df['volume'], length=20)
df['vwap'] = ta.vwap(df['high'], df['low'], df['close'], df['volume'])
df['vwap_open'] = df['vwap'] / df['open']
df['rsi'] = ta.rsi(df['close'], length=14)
bb = ta.bbands(df['close'], length=20)
df['bb_width'] = (bb['BBU_20_2.0'] - bb['BBL_20_2.0']) / df['close']
df['volume_change'] = df['volume'].pct_change()

# Исправление MACD
macd = ta.macd(df['close'], fast=12, slow=26, signal=9)
print("Доступные столбцы MACD:", macd.columns)  # Проверка имен столбцов
df['macd'] = macd['MACD_12_26_9']
df['macd_signal'] = macd['MACDs_12_26_9']  # Исправлено
df['adx'] = ta.adx(df['high'], df['low'], df['close'], length=14)['ADX_14']

# Разделение данных
train_df = df['2017-08-23 16:00:00':'2021-12-31 16:00:00']
val_df = df['2022-01-01 00:00:00':'2022-02-13 20:00:00']
test_df = df['2022-02-14 00:00:00':'2022-04-23 00:00:00']

# Ограничение признаков
key_features = ['open', 'high', 'low', 'close', 'volume', 'close_open', 'high_low', 
                'open_diff_1', 'open_diff_2', 'close_diff_1', 'close_diff_2', 
                'volume_diff_1', 'volume_diff_2', 'cmf', 'vwap_open', 'atr', 
                'rsi', 'bb_width', 'volume_change', 'macd', 'macd_signal', 'adx']
features = [col for col in df.columns if col in key_features]
X_train = train_df[features].values
y_train = train_df['target'].values
X_val = val_df[features].values
y_val = val_df['target'].values
X_test = test_df[features].values
y_test = test_df['target'].values

# Обработка inf и NaN
for i, col in enumerate(features):
    max_val = np.percentile(df[col][~np.isinf(df[col])], 99)
    min_val = np.percentile(df[col][~np.isinf(df[col])], 1)
    X_train[:, i] = np.where(np.isinf(X_train[:, i]), max_val, X_train[:, i])
    X_val[:, i] = np.where(np.isinf(X_val[:, i]), max_val, X_val[:, i])
    X_test[:, i] = np.where(np.isinf(X_test[:, i]), max_val, X_test[:, i])
X_train = np.nan_to_num(X_train, nan=0.0)
X_val = np.nan_to_num(X_val, nan=0.0)
X_test = np.nan_to_num(X_test, nan=0.0)

# Нормализация и шум
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train) + np.random.normal(0, 0.01, X_train.shape)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

# Oversampling с SMOTE
smote = SMOTE(sampling_strategy={1: 4000, 2: 3000}, random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

# Проверка распределения
print("Распределение y_train (после SMOTE):", np.bincount(y_train_res))
print("Распределение y_val:", np.bincount(y_val))
print("Распределение y_test:", np.bincount(y_test))

# Веса классов
class_weights_dict = {0: 1.0, 1: 1.2, 2: 2.0}
print("Веса классов:", class_weights_dict)

# Инициализация и обучение TabNet
clf = TabNetClassifier(
    n_d=128, n_a=128, n_steps=7, gamma=1.5,
    lambda_sparse=0.001, optimizer_params=dict(lr=1e-3, weight_decay=1e-5),
    mask_type='sparsemax', n_shared=2,
    verbose=1, seed=42
)
clf.fit(
    X_train_res, y_train_res,
    eval_set=[(X_val, y_val)],
    eval_metric=['accuracy', 'balanced_accuracy'],
    max_epochs=300,
    patience=100,
    batch_size=512,
    virtual_batch_size=256,
    weights=class_weights_dict,
    drop_last=True
)

# Предсказание и оценка
y_pred = clf.predict(X_test)
print("Распределение предсказанных классов:", np.bincount(y_pred))
print("Распределение истинных классов:", np.bincount(y_test))
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')
print(f"Accuracy: {accuracy:.4f}")
print(f"F1-score: {f1:.4f}")
print("\nClassification Report:\n", classification_report(y_test, y_pred, target_names=['c0', 'c1', 'c2']))

# Важность признаков
feature_importances = pd.Series(clf.feature_importances_, index=features)
print("\nТоп-5 важных признаков:\n", feature_importances.sort_values(ascending=False).head())

Доступные столбцы MACD: Index(['MACD_12_26_9', 'MACDh_12_26_9', 'MACDs_12_26_9'], dtype='object')
Распределение y_train (после SMOTE): [5519 4000 3000]
Распределение y_val: [149  92  23]
Распределение y_test: [219 126  64]
Веса классов: {0: 1.0, 1: 1.2, 2: 2.0}


/usr/local/lib/python3.12/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 2.02174 | val_0_accuracy: 0.49621 | val_0_balanced_accuracy: 0.32611 |  0:00:01s
epoch 1  | loss: 1.57127 | val_0_accuracy: 0.43939 | val_0_balanced_accuracy: 0.31196 |  0:00:03s
epoch 2  | loss: 1.30817 | val_0_accuracy: 0.42803 | val_0_balanced_accuracy: 0.34501 |  0:00:04s
epoch 3  | loss: 1.18249 | val_0_accuracy: 0.49621 | val_0_balanced_accuracy: 0.38899 |  0:00:05s
epoch 4  | loss: 1.17537 | val_0_accuracy: 0.34091 | val_0_balanced_accuracy: 0.38861 |  0:00:07s
epoch 5  | loss: 1.15593 | val_0_accuracy: 0.4053  | val_0_balanced_accuracy: 0.3302  |  0:00:08s
epoch 6  | loss: 1.12661 | val_0_accuracy: 0.43939 | val_0_balanced_accuracy: 0.37484 |  0:00:10s
epoch 7  | loss: 1.05303 | val_0_accuracy: 0.49242 | val_0_balanced_accuracy: 0.39157 |  0:00:11s
epoch 8  | loss: 1.05431 | val_0_accuracy: 0.41667 | val_0_balanced_accuracy: 0.32232 |  0:00:13s
epoch 9  | loss: 1.06053 | val_0_accuracy: 0.42424 | val_0_balanced_accuracy: 0.34598 |  0:00:14s
epoch 10 | loss: 1.0

/usr/local/lib/python3.12/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Распределение предсказанных классов: [142 223  44]
Распределение истинных классов: [219 126  64]
Accuracy: 0.4034
F1-score: 0.4080

Classification Report:
               precision    recall  f1-score   support

          c0       0.59      0.38      0.47       219
          c1       0.30      0.53      0.38       126
          c2       0.32      0.22      0.26        64

    accuracy                           0.40       409
   macro avg       0.40      0.38      0.37       409
weighted avg       0.46      0.40      0.41       409


Топ-5 важных признаков:
 high_low       0.149078
open_diff_2    0.093540
rsi            0.068282
atr            0.058775
open           0.057926
dtype: float64


In [15]:
from pytorch_tabnet.tab_model import TabNetClassifier
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, classification_report
from imblearn.over_sampling import SMOTE
import pandas_ta as ta
import pandas as pd
from sklearn.preprocessing import StandardScaler
import torch

# Загрузка и обработка данных
df = pd.read_csv('btc_usdt_4h_with_target.csv', index_col='timestamp', parse_dates=True)
df.ffill(inplace=True)
df.fillna(0, inplace=True)

# Пересчет целевой переменной
df['high_next'] = df['high'].shift(-1)
df['low_next'] = df['low'].shift(-1)
df['u_t1'] = (df['high_next'] - df['close']) / df['close']
df['v_t1'] = (df['low_next'] - df['close']) / df['close']
threshold = 0.006
df['target'] = np.where(df['u_t1'] >= threshold, 0, np.where(df['v_t1'] <= -threshold, 1, 2))

# Добавление признаков
def calculate_atr(df, period=14):
    df['tr'] = np.maximum(df['high'] - df['low'], 
                         np.maximum(abs(df['high'] - df['close'].shift(1)), 
                                    abs(df['low'] - df['close'].shift(1))))
    df['atr'] = df['tr'].rolling(window=period).mean()
    return df

df = calculate_atr(df)
df['cmf'] = ta.cmf(df['high'], df['low'], df['close'], df['volume'], length=20)
df['vwap'] = ta.vwap(df['high'], df['low'], df['close'], df['volume'])
df['vwap_open'] = df['vwap'] / df['open']
df['rsi'] = ta.rsi(df['close'], length=14)
bb = ta.bbands(df['close'], length=20)
df['bb_width'] = (bb['BBU_20_2.0'] - bb['BBL_20_2.0']) / df['close']
df['volume_change'] = df['volume'].pct_change()
macd = ta.macd(df['close'], fast=12, slow=26, signal=9)
df['macd'] = macd['MACD_12_26_9']
df['macd_signal'] = macd['MACDs_12_26_9']
df['adx'] = ta.adx(df['high'], df['low'], df['close'], length=14)['ADX_14']
df['ema_short'] = ta.ema(df['close'], length=12)
df['ema_long'] = ta.ema(df['close'], length=50)
df['atr_norm'] = df['atr'] / df['close']
df['log_close'] = np.log1p(df['close'])
df['ema_ratio'] = df['ema_short'] / df['ema_long']
stoch = ta.stoch(df['high'], df['low'], df['close'], k=14, d=3, smooth_k=3)
df['stoch_k'] = stoch['STOCHk_14_3_3']
df['stoch_d'] = stoch['STOCHd_14_3_3']
df['obv'] = ta.obv(df['close'], df['volume'])
df['cci'] = ta.cci(df['high'], df['low'], df['close'], length=20)

# Разделение данных
train_df = df['2017-08-23 16:00:00':'2021-12-31 16:00:00']
val_df = df['2022-01-01 00:00:00':'2022-02-13 20:00:00']
test_df = df['2022-02-14 00:00:00':'2022-04-23 00:00:00']

# Ограничение признаков
key_features = ['open', 'high', 'low', 'close', 'volume', 'close_open', 'high_low', 
                'open_diff_1', 'open_diff_2', 'close_diff_1', 'close_diff_2', 
                'volume_diff_1', 'volume_diff_2', 'cmf', 'vwap_open', 'atr', 
                'rsi', 'bb_width', 'volume_change', 'macd', 'macd_signal', 'adx',
                'ema_short', 'ema_long', 'atr_norm', 'log_close', 'ema_ratio',
                'stoch_k', 'stoch_d', 'obv', 'cci']
features = [col for col in df.columns if col in key_features]
X_train = train_df[features].values
y_train = train_df['target'].values
X_val = val_df[features].values
y_val = val_df['target'].values
X_test = test_df[features].values
y_test = test_df['target'].values

# Обработка inf и NaN
for i, col in enumerate(features):
    max_val = np.percentile(df[col][~np.isinf(df[col])], 99)
    min_val = np.percentile(df[col][~np.isinf(df[col])], 1)
    X_train[:, i] = np.where(np.isinf(X_train[:, i]), max_val, X_train[:, i])
    X_val[:, i] = np.where(np.isinf(X_val[:, i]), max_val, X_val[:, i])
    X_test[:, i] = np.where(np.isinf(X_test[:, i]), max_val, X_test[:, i])
X_train = np.nan_to_num(X_train, nan=0.0)
X_val = np.nan_to_num(X_val, nan=0.0)
X_test = np.nan_to_num(X_test, nan=0.0)

# Нормализация и шум
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train) + np.random.normal(0, 0.01, X_train.shape)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

# Oversampling с SMOTE
smote = SMOTE(sampling_strategy={1: 3000, 2: 3000}, random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

# Проверка распределения
print("Распределение y_train (после SMOTE):", np.bincount(y_train_res))
print("Распределение y_val:", np.bincount(y_val))
print("Распределение y_test:", np.bincount(y_test))

# Веса классов
class_weights_dict = {0: 1.0, 1: 1.0, 2: 2.0}
print("Веса классов:", class_weights_dict)

# Инициализация и обучение TabNet
clf = TabNetClassifier(
    n_d=512, n_a=512, n_steps=7, gamma=1.5,
    lambda_sparse=0.0005, 
    optimizer_params=dict(lr=1e-3, weight_decay=1e-5),
    scheduler_fn=torch.optim.lr_scheduler.StepLR,
    scheduler_params={"step_size": 30, "gamma": 0.85},
    mask_type='sparsemax', n_shared=2,
    verbose=1, seed=42
)
clf.fit(
    X_train_res, y_train_res,
    eval_set=[(X_val, y_val)],
    eval_metric=['accuracy', 'balanced_accuracy'],
    max_epochs=300,
    patience=100,
    batch_size=1024,
    virtual_batch_size=512,
    weights=class_weights_dict,
    drop_last=True
)

# Предсказание и оценка
y_pred = clf.predict(X_test)
print("Распределение предсказанных классов:", np.bincount(y_pred))
print("Распределение истинных классов:", np.bincount(y_test))
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')
print(f"Accuracy: {accuracy:.4f}")
print(f"F1-score: {f1:.4f}")
print("\nClassification Report:\n", classification_report(y_test, y_pred, target_names=['c0', 'c1', 'c2']))

# Важность признаков
feature_importances = pd.Series(clf.feature_importances_, index=features)
print("\nТоп-5 важных признаков:\n", feature_importances.sort_values(ascending=False).head())

Распределение y_train (после SMOTE): [5519 3000 3000]
Распределение y_val: [149  92  23]
Распределение y_test: [219 126  64]
Веса классов: {0: 1.0, 1: 1.0, 2: 2.0}


/usr/local/lib/python3.12/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 4.4802  | val_0_accuracy: 0.45455 | val_0_balanced_accuracy: 0.32346 |  0:00:01s
epoch 1  | loss: 1.85568 | val_0_accuracy: 0.51515 | val_0_balanced_accuracy: 0.33591 |  0:00:02s
epoch 2  | loss: 1.71389 | val_0_accuracy: 0.49621 | val_0_balanced_accuracy: 0.38878 |  0:00:03s
epoch 3  | loss: 1.51232 | val_0_accuracy: 0.49242 | val_0_balanced_accuracy: 0.3292  |  0:00:04s
epoch 4  | loss: 1.6165  | val_0_accuracy: 0.47727 | val_0_balanced_accuracy: 0.33294 |  0:00:05s
epoch 5  | loss: 1.53797 | val_0_accuracy: 0.40152 | val_0_balanced_accuracy: 0.37239 |  0:00:06s
epoch 6  | loss: 1.64364 | val_0_accuracy: 0.30303 | val_0_balanced_accuracy: 0.25477 |  0:00:07s
epoch 7  | loss: 1.71464 | val_0_accuracy: 0.39394 | val_0_balanced_accuracy: 0.30248 |  0:00:08s
epoch 8  | loss: 1.67663 | val_0_accuracy: 0.52273 | val_0_balanced_accuracy: 0.44397 |  0:00:09s
epoch 9  | loss: 1.65991 | val_0_accuracy: 0.48106 | val_0_balanced_accuracy: 0.36341 |  0:00:10s
epoch 10 | loss: 1.6

/usr/local/lib/python3.12/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Распределение предсказанных классов: [288  54  67]
Распределение истинных классов: [219 126  64]
Accuracy: 0.4768
F1-score: 0.4420

Classification Report:
               precision    recall  f1-score   support

          c0       0.56      0.74      0.64       219
          c1       0.37      0.16      0.22       126
          c2       0.21      0.22      0.21        64

    accuracy                           0.48       409
   macro avg       0.38      0.37      0.36       409
weighted avg       0.45      0.48      0.44       409


Топ-5 важных признаков:
 stoch_d         0.085833
high_low        0.064700
ema_long        0.055856
bb_width        0.052458
close_diff_2    0.049132
dtype: float64


In [19]:
from pytorch_tabnet.tab_model import TabNetClassifier
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, classification_report
from imblearn.over_sampling import SMOTE
import pandas_ta as ta
import pandas as pd
from sklearn.preprocessing import StandardScaler
import torch

# Загрузка и обработка данных
df = pd.read_csv('btc_usdt_4h_with_target.csv', index_col='timestamp', parse_dates=True)
df.ffill(inplace=True)
df.fillna(0, inplace=True)

# Пересчет целевой переменной
df['high_next'] = df['high'].shift(-1)
df['low_next'] = df['low'].shift(-1)
df['u_t1'] = (df['high_next'] - df['close']) / df['close']
df['v_t1'] = (df['low_next'] - df['close']) / df['close']
threshold = 0.006
df['target'] = np.where(df['u_t1'] >= threshold, 0, np.where(df['v_t1'] <= -threshold, 1, 2))

# Добавление признаков
def calculate_atr(df, period=14):
    df['tr'] = np.maximum(df['high'] - df['low'], 
                         np.maximum(abs(df['high'] - df['close'].shift(1)), 
                                    abs(df['low'] - df['close'].shift(1))))
    df['atr'] = df['tr'].rolling(window=period).mean()
    return df

df = calculate_atr(df)
df['cmf'] = ta.cmf(df['high'], df['low'], df['close'], df['volume'], length=20)
df['vwap'] = ta.vwap(df['high'], df['low'], df['close'], df['volume'])
df['vwap_open'] = df['vwap'] / df['open']
df['rsi'] = ta.rsi(df['close'], length=14)
bb = ta.bbands(df['close'], length=20)
df['bb_width'] = (bb['BBU_20_2.0'] - bb['BBL_20_2.0']) / df['close']
df['volume_change'] = df['volume'].pct_change()
macd = ta.macd(df['close'], fast=12, slow=26, signal=9)
df['macd'] = macd['MACD_12_26_9']
df['macd_signal'] = macd['MACDs_12_26_9']
df['adx'] = ta.adx(df['high'], df['low'], df['close'], length=14)['ADX_14']
df['ema_short'] = ta.ema(df['close'], length=12)
df['ema_long'] = ta.ema(df['close'], length=50)
df['atr_norm'] = df['atr'] / df['close']
df['ema_ratio'] = df['ema_short'] / df['ema_long']
stoch = ta.stoch(df['high'], df['low'], df['close'], k=14, d=3, smooth_k=3)
df['stoch_d'] = stoch['STOCHd_14_3_3']
df['obv'] = ta.obv(df['close'], df['volume'])
df['cci'] = ta.cci(df['high'], df['low'], df['close'], length=20)
df['roc'] = ta.roc(df['close'], length=10)
df['volume_diff_3'] = df['volume'].diff(3)

# Разделение данных
train_df = df['2017-08-23 16:00:00':'2021-12-31 16:00:00']
val_df = df['2022-01-01 00:00:00':'2022-02-13 20:00:00']
test_df = df['2022-02-14 00:00:00':'2022-04-23 00:00:00']

# Ограничение признаков
key_features = ['open', 'high', 'low', 'close', 'volume', 'close_open', 'high_low', 
                'open_diff_1', 'open_diff_2', 'close_diff_1', 'close_diff_2', 
                'volume_diff_1', 'volume_diff_2', 'cmf', 'vwap_open', 'atr', 
                'rsi', 'bb_width', 'volume_change', 'macd', 'macd_signal', 'adx',
                'ema_short', 'ema_long', 'atr_norm', 'ema_ratio', 'stoch_d', 
                'obv', 'cci', 'roc', 'volume_diff_3']
features = [col for col in df.columns if col in key_features]
X_train = train_df[features].values
y_train = train_df['target'].values
X_val = val_df[features].values
y_val = val_df['target'].values
X_test = test_df[features].values
y_test = test_df['target'].values

# Обработка inf и NaN
for i, col in enumerate(features):
    max_val = np.percentile(df[col][~np.isinf(df[col])], 99)
    min_val = np.percentile(df[col][~np.isinf(df[col])], 1)
    X_train[:, i] = np.where(np.isinf(X_train[:, i]), max_val, X_train[:, i])
    X_val[:, i] = np.where(np.isinf(X_val[:, i]), max_val, X_val[:, i])
    X_test[:, i] = np.where(np.isinf(X_test[:, i]), max_val, X_test[:, i])
X_train = np.nan_to_num(X_train, nan=0.0)
X_val = np.nan_to_num(X_val, nan=0.0)
X_test = np.nan_to_num(X_test, nan=0.0)

# Нормализация и шум
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train) + np.random.normal(0, 0.02, X_train.shape)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

# Oversampling с SMOTE
smote = SMOTE(sampling_strategy={1: 4000, 2: 3000}, random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

# Проверка распределения
print("Распределение y_train (после SMOTE):", np.bincount(y_train_res))
print("Распределение y_val:", np.bincount(y_val))
print("Распределение y_test:", np.bincount(y_test))

# Веса классов
class_weights_dict = {0: 1.0, 1: 1.2, 2: 2.0}
print("Веса классов:", class_weights_dict)

# Инициализация и обучение TabNet
clf = TabNetClassifier(
    n_d=256, n_a=256, n_steps=8, gamma=1.5,
    lambda_sparse=0.0005, 
    optimizer_params=dict(lr=1e-3, weight_decay=1e-5),
    scheduler_fn=torch.optim.lr_scheduler.StepLR,
    scheduler_params={"step_size": 50, "gamma": 0.9},
    mask_type='sparsemax', n_shared=2,
    verbose=1, seed=42
)
clf.fit(
    X_train_res, y_train_res,
    eval_set=[(X_val, y_val)],
    eval_metric=['accuracy', 'balanced_accuracy'],
    max_epochs=300,
    patience=50,
    batch_size=1024,
    virtual_batch_size=512,
    weights=class_weights_dict,
    drop_last=True
)

# Предсказание и оценка
y_pred = clf.predict(X_test)
print("Распределение предсказанных классов:", np.bincount(y_pred))
print("Распределение истинных классов:", np.bincount(y_test))
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')
print(f"Accuracy: {accuracy:.4f}")
print(f"F1-score: {f1:.4f}")
print("\nClassification Report:\n", classification_report(y_test, y_pred, target_names=['c0', 'c1', 'c2']))

# Важность признаков
feature_importances = pd.Series(clf.feature_importances_, index=features)
print("\nТоп-5 важных признаков:\n", feature_importances.sort_values(ascending=False).head())

Распределение y_train (после SMOTE): [5519 4000 3000]
Распределение y_val: [149  92  23]
Распределение y_test: [219 126  64]
Веса классов: {0: 1.0, 1: 1.2, 2: 2.0}


/usr/local/lib/python3.12/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 2.95854 | val_0_accuracy: 0.29924 | val_0_balanced_accuracy: 0.34503 |  0:00:01s
epoch 1  | loss: 2.14501 | val_0_accuracy: 0.43561 | val_0_balanced_accuracy: 0.3416  |  0:00:02s
epoch 2  | loss: 1.79233 | val_0_accuracy: 0.32955 | val_0_balanced_accuracy: 0.32499 |  0:00:03s
epoch 3  | loss: 1.57092 | val_0_accuracy: 0.35985 | val_0_balanced_accuracy: 0.36623 |  0:00:03s
epoch 4  | loss: 1.42925 | val_0_accuracy: 0.37879 | val_0_balanced_accuracy: 0.36998 |  0:00:04s
epoch 5  | loss: 1.28766 | val_0_accuracy: 0.44318 | val_0_balanced_accuracy: 0.38977 |  0:00:05s
epoch 6  | loss: 1.26701 | val_0_accuracy: 0.42045 | val_0_balanced_accuracy: 0.34352 |  0:00:06s
epoch 7  | loss: 1.32903 | val_0_accuracy: 0.4697  | val_0_balanced_accuracy: 0.42301 |  0:00:07s
epoch 8  | loss: 1.31074 | val_0_accuracy: 0.38636 | val_0_balanced_accuracy: 0.36825 |  0:00:08s
epoch 9  | loss: 1.29672 | val_0_accuracy: 0.38258 | val_0_balanced_accuracy: 0.31933 |  0:00:09s
epoch 10 | loss: 1.3

/usr/local/lib/python3.12/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Распределение предсказанных классов: [210 105  94]
Распределение истинных классов: [219 126  64]
Accuracy: 0.3985
F1-score: 0.4031

Classification Report:
               precision    recall  f1-score   support

          c0       0.56      0.53      0.55       219
          c1       0.28      0.23      0.25       126
          c2       0.18      0.27      0.22        64

    accuracy                           0.40       409
   macro avg       0.34      0.34      0.34       409
weighted avg       0.41      0.40      0.40       409


Топ-5 важных признаков:
 close       0.074021
low         0.062543
adx         0.055790
high        0.053958
high_low    0.047183
dtype: float64


In [20]:
from pytorch_tabnet.tab_model import TabNetClassifier
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, classification_report
from imblearn.over_sampling import SMOTE
import pandas_ta as ta
import pandas as pd
from sklearn.preprocessing import StandardScaler
import torch

# Загрузка и обработка данных
df = pd.read_csv('btc_usdt_4h_with_target.csv', index_col='timestamp', parse_dates=True)
df.ffill(inplace=True)
df.fillna(0, inplace=True)

# Пересчет целевой переменной
df['high_next'] = df['high'].shift(-1)
df['low_next'] = df['low'].shift(-1)
df['u_t1'] = (df['high_next'] - df['close']) / df['close']
df['v_t1'] = (df['low_next'] - df['close']) / df['close']
threshold = 0.006
df['target'] = np.where(df['u_t1'] >= threshold, 0, np.where(df['v_t1'] <= -threshold, 1, 2))

# Добавление признаков
def calculate_atr(df, period=14):
    df['tr'] = np.maximum(df['high'] - df['low'], 
                         np.maximum(abs(df['high'] - df['close'].shift(1)), 
                                    abs(df['low'] - df['close'].shift(1))))
    df['atr'] = df['tr'].rolling(window=period).mean()
    return df

df = calculate_atr(df)
df['cmf'] = ta.cmf(df['high'], df['low'], df['close'], df['volume'], length=20)
df['vwap'] = ta.vwap(df['high'], df['low'], df['close'], df['volume'])
df['vwap_open'] = df['vwap'] / df['open']
df['rsi'] = ta.rsi(df['close'], length=14)
bb = ta.bbands(df['close'], length=20)
df['bb_width'] = (bb['BBU_20_2.0'] - bb['BBL_20_2.0']) / df['close']
df['volume_change'] = df['volume'].pct_change()
macd = ta.macd(df['close'], fast=12, slow=26, signal=9)
df['macd'] = macd['MACD_12_26_9']
df['macd_signal'] = macd['MACDs_12_26_9']
df['adx'] = ta.adx(df['high'], df['low'], df['close'], length=14)['ADX_14']
df['ema_short'] = ta.ema(df['close'], length=12)
df['ema_long'] = ta.ema(df['close'], length=50)
df['atr_norm'] = df['atr'] / df['close']
df['ema_ratio'] = df['ema_short'] / df['ema_long']
stoch = ta.stoch(df['high'], df['low'], df['close'], k=14, d=3, smooth_k=3)
df['stoch_d'] = stoch['STOCHd_14_3_3']
df['obv'] = ta.obv(df['close'], df['volume'])
df['cci'] = ta.cci(df['high'], df['low'], df['close'], length=20)
df['roc'] = ta.roc(df['close'], length=10)
df['atr_diff'] = df['atr'].diff(1)
df['momentum'] = ta.mom(df['close'], length=10)

# Разделение данных
train_df = df['2017-08-23 16:00:00':'2021-12-31 16:00:00']
val_df = df['2022-01-01 00:00:00':'2022-02-13 20:00:00']
test_df = df['2022-02-14 00:00:00':'2022-04-23 00:00:00']

# Ограничение признаков
key_features = ['open', 'high', 'low', 'close', 'volume', 'close_open', 'high_low', 
                'open_diff_1', 'open_diff_2', 'close_diff_1', 'close_diff_2', 
                'volume_diff_1', 'volume_diff_2', 'cmf', 'vwap_open', 'atr', 
                'rsi', 'bb_width', 'volume_change', 'macd', 'macd_signal', 'adx',
                'ema_short', 'ema_long', 'atr_norm', 'ema_ratio', 'stoch_d', 
                'obv', 'cci', 'roc', 'atr_diff', 'momentum']
features = [col for col in df.columns if col in key_features]
X_train = train_df[features].values
y_train = train_df['target'].values
X_val = val_df[features].values
y_val = val_df['target'].values
X_test = test_df[features].values
y_test = test_df['target'].values

# Обработка inf и NaN
for i, col in enumerate(features):
    max_val = np.percentile(df[col][~np.isinf(df[col])], 99)
    min_val = np.percentile(df[col][~np.isinf(df[col])], 1)
    X_train[:, i] = np.where(np.isinf(X_train[:, i]), max_val, X_train[:, i])
    X_val[:, i] = np.where(np.isinf(X_val[:, i]), max_val, X_val[:, i])
    X_test[:, i] = np.where(np.isinf(X_test[:, i]), max_val, X_test[:, i])
X_train = np.nan_to_num(X_train, nan=0.0)
X_val = np.nan_to_num(X_val, nan=0.0)
X_test = np.nan_to_num(X_test, nan=0.0)

# Нормализация и шум
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train) + np.random.normal(0, 0.015, X_train.shape)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

# Oversampling с SMOTE
smote = SMOTE(sampling_strategy={1: 3500, 2: 3500}, random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

# Проверка распределения
print("Распределение y_train (после SMOTE):", np.bincount(y_train_res))
print("Распределение y_val:", np.bincount(y_val))
print("Распределение y_test:", np.bincount(y_test))

# Веса классов
class_weights_dict = {0: 1.2, 1: 1.2, 2: 2.0}
print("Веса классов:", class_weights_dict)

# Инициализация и обучение TabNet
clf = TabNetClassifier(
    n_d=128, n_a=128, n_steps=6, gamma=1.5,
    lambda_sparse=0.0005, 
    optimizer_params=dict(lr=1e-3, weight_decay=1e-5),
    scheduler_fn=torch.optim.lr_scheduler.StepLR,
    scheduler_params={"step_size": 75, "gamma": 0.95},
    mask_type='sparsemax', n_shared=2,
    verbose=1, seed=42
)
clf.fit(
    X_train_res, y_train_res,
    eval_set=[(X_val, y_val)],
    eval_metric=['accuracy', 'balanced_accuracy'],
    max_epochs=300,
    patience=75,
    batch_size=1024,
    virtual_batch_size=512,
    weights=class_weights_dict,
    drop_last=True
)

# Предсказание и оценка
y_pred = clf.predict(X_test)
print("Распределение предсказанных классов:", np.bincount(y_pred))
print("Распределение истинных классов:", np.bincount(y_test))
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')
print(f"Accuracy: {accuracy:.4f}")
print(f"F1-score: {f1:.4f}")
print("\nClassification Report:\n", classification_report(y_test, y_pred, target_names=['c0', 'c1', 'c2']))

# Важность признаков
feature_importances = pd.Series(clf.feature_importances_, index=features)
print("\nТоп-5 важных признаков:\n", feature_importances.sort_values(ascending=False).head())

Распределение y_train (после SMOTE): [5519 3500 3500]
Распределение y_val: [149  92  23]
Распределение y_test: [219 126  64]
Веса классов: {0: 1.2, 1: 1.2, 2: 2.0}


/usr/local/lib/python3.12/dist-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 2.40339 | val_0_accuracy: 0.375   | val_0_balanced_accuracy: 0.32871 |  0:00:00s
epoch 1  | loss: 1.61511 | val_0_accuracy: 0.5303  | val_0_balanced_accuracy: 0.37674 |  0:00:01s
epoch 2  | loss: 1.3987  | val_0_accuracy: 0.4697  | val_0_balanced_accuracy: 0.33657 |  0:00:02s
epoch 3  | loss: 1.25687 | val_0_accuracy: 0.48106 | val_0_balanced_accuracy: 0.32431 |  0:00:02s
epoch 4  | loss: 1.15569 | val_0_accuracy: 0.53409 | val_0_balanced_accuracy: 0.36512 |  0:00:03s
epoch 5  | loss: 1.10847 | val_0_accuracy: 0.46591 | val_0_balanced_accuracy: 0.33316 |  0:00:03s
epoch 6  | loss: 1.02615 | val_0_accuracy: 0.47348 | val_0_balanced_accuracy: 0.34413 |  0:00:04s
epoch 7  | loss: 1.04261 | val_0_accuracy: 0.53409 | val_0_balanced_accuracy: 0.36884 |  0:00:05s
epoch 8  | loss: 1.00917 | val_0_accuracy: 0.42045 | val_0_balanced_accuracy: 0.34425 |  0:00:05s
epoch 9  | loss: 1.00953 | val_0_accuracy: 0.4053  | val_0_balanced_accuracy: 0.31313 |  0:00:06s
epoch 10 | loss: 0.9

/usr/local/lib/python3.12/dist-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Распределение предсказанных классов: [250  43 116]
Распределение истинных классов: [219 126  64]
Accuracy: 0.4425
F1-score: 0.4106

Classification Report:
               precision    recall  f1-score   support

          c0       0.56      0.64      0.60       219
          c1       0.23      0.08      0.12       126
          c2       0.26      0.47      0.33        64

    accuracy                           0.44       409
   macro avg       0.35      0.40      0.35       409
weighted avg       0.41      0.44      0.41       409


Топ-5 важных признаков:
 atr_norm     0.115579
bb_width     0.059480
ema_long     0.058632
vwap_open    0.053979
atr          0.048959
dtype: float64
